In [14]:
import pickle
from collections import defaultdict
from typing import Sequence

from llama_index.core import Settings
from llama_index.core.schema import BaseNode, MetadataMode


In [14]:
with open('C:/Agentes-RAG/src/data/nodes/Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.temporal2.pk', 'rb') as file:
    model:Sequence[BaseNode] = pickle.load(file)

In [15]:
print(model[4].get_content(MetadataMode.ALL))
print(len(model))

block_type: Author Names
file_type: application/pdf
languages: ['eng']
page_number: 1
file_name: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf
title_of_the_document: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
image_path: nan
makes_sense: True
description: Lists the authors of the document.

Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
11


In [89]:
from llama_index.core.schema import NodeRelationship

node: BaseNode = model[10]
print(node.node_id)
print(node.text)
print(node.metadata['block_type'])
print(node.metadata['image_path'])
node.relationships[NodeRelationship.PREVIOUS].node_id
res = list(filter(
    lambda x: x.node_id == '0_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', 
    model
))
# if len(res) > 0:
#     print(res[0].text)
node.metadata

res

5_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Author Names
nan


[Document(id_='0_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', embedding=None, metadata={'block_type': 'UncategorizedText', 'file_type': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_name': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'title_of_the_document': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', 'image_path': nan, 'makes_sense': False, 'description': 'The text block contains a sequence of numbers without any context or explanation, making it nonsensical.'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', node_type=None, metadata={}, hash=None)}, text='1 2 0 2', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]

In [59]:
node.metadata['makes_sense'] = True
node.metadata['title_of_the_document']

'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'

In [99]:
res[0].node_id, res[0].relationships.get(NodeRelationship.PREVIOUS).node_id

AttributeError: 'NoneType' object has no attribute 'node_id'

In [63]:
from src.Ingestion.lib.library_custom.llama_index.transformations import Unstructured_Medatata_PostProcessor
tranformation = Unstructured_Medatata_PostProcessor()
model[3:5]

[Document(id_='3_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', embedding=None, metadata={'block_type': 'Title', 'file_type': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'file_name': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'title_of_the_document': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', 'image_path': nan}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='2_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', node_type=None, metadata={}, hash=None), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='4_Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', node_type=None, metadata={}, hash=None)}, text='Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', me

In [1]:
import os
os.environ['GCP_CREDENTIALS_PATH'] = r"C:\Agentes-RAG\GCP-Credentials\llms-433815-5e7ca2a0c045.json"

from src.app.config import Config
# Config.set_gcp_llm_by_name('gemini-1.5-flash-001')
print(Config.llm.model)
# llm = Config.llm
# print(llm.complete('Hola').text)

from src.app.agent.load_index import NodeLoaderAgent

indx_agent = NodeLoaderAgent(
    pkl_path='C:/Agentes-RAG/GCP/Procesar Bloques Unstructured/embeded_nodes_v2.pkl',
    verbose=False
)

gemini-1.5-pro-001


In [8]:
sq = indx_agent.summary_index
ret = sq.as_retriever().retrieve('Give me a summary')
print(len(ret))

287


In [2]:
qe = indx_agent.summary_qe
qe

In [10]:
res = qe.query('Give me a summary') # ventana de contexto por defecto
res.response

'The retrieval component would "collapse" and learn to retrieve the same documents regardless of the input for tasks such as story generation [11]. \n'

In [3]:
res = qe.query('Give me a summary') # ventana de contexto de 1 millon de tokens
res.response

'This research paper explores Retrieval-Augmented Generation (RAG) models, which combine pre-trained language models with access to external knowledge. The authors demonstrate that RAG models outperform other approaches on tasks like question answering and fact verification. They also highlight the advantages of RAG models, such as their flexibility and ability to update their knowledge easily. The paper discusses the technical details of RAG models, including their components, training process, and decoding strategies. It also explores the challenges of using external knowledge sources and the potential societal impact of these models. \n'

In [3]:
res = qe.query('Give me a summary') # ventana de contexto de 1 millon de tokens Gemini 1.5 Pro
res.response

"This document presents research on Retrieval-Augmented Generation (RAG) models for knowledge-intensive NLP tasks. RAG models combine pre-trained parametric and non-parametric memory for language generation, achieving state-of-the-art results on various tasks, including open-domain question answering, abstractive question answering, Jeopardy question generation, and fact verification. The authors explore two RAG formulations: RAG-Sequence and RAG-Token, and demonstrate their effectiveness through extensive experiments and comparisons with existing models. The research highlights the benefits of combining parametric and non-parametric memory, showcasing RAG's ability to generate more factual, specific, and diverse language while mitigating issues like hallucination. The authors also discuss the flexibility of RAG models in updating knowledge and explore potential societal benefits and drawbacks. \n"

In [12]:
res.response

'The retrieval component would "collapse" and learn to retrieve the same documents regardless of the input for tasks such as story generation [11]. \n'

In [15]:
Settings.context_window

3900

In [18]:
Settings.llm.metadata

LLMMetadata(context_window=3900, num_output=256, is_chat_model=True, is_function_calling_model=True, model_name='gemini-1.5-flash-001', system_role=<MessageRole.USER: 'user'>)

In [20]:
Config.llm.complete("""Construct a very short prompt that accurately describes the use of this information retrieval tool for performing Query Answering (QA) tasks. This tool contains a document with the following information:
    <document>
    This document presents research on Retrieval-Augmented Generation (RAG) models for knowledge-intensive NLP tasks. RAG models combine pre-trained parametric and non-parametric memory for language generation, achieving state-of-the-art results on various tasks, including open-domain question answering, abstractive question answering, Jeopardy question generation, and fact verification. The authors explore two RAG formulations: RAG-Sequence and RAG-Token, and demonstrate their effectiveness through extensive experiments and comparisons with existing models. The research highlights the benefits of combining parametric and non-parametric memory, showcasing RAG's ability to generate more factual, specific, and diverse language while mitigating issues like hallucination. The authors also discuss the flexibility of RAG models in updating knowledge and explore potential societal benefits and drawbacks.
    </document>
The description should be brief and primarily based on the following example:
    <example_prompt>
    Useful for any requests that require a holistic summary of EVERYTHING about California. For questions  more specific sections, please use the vector_tool.about
    </example_prompt>
Additionally, include minimal but relevant information about the tool itself, such as how it works and specific examples of when it would be useful, particularly in the context of the document.""").text

"**Prompt:**\n\nUseful for answering questions about **Retrieval-Augmented Generation (RAG) models** and their applications in **knowledge-intensive NLP tasks**. \n\n**Explanation:**\n\nThis tool utilizes a document summarizing research on RAG models. It's particularly helpful for questions about how RAG models combine different memory types for language generation, their effectiveness in tasks like **question answering and fact verification**, and their potential benefits and drawbacks. For questions about specific RAG formulations (e.g., RAG-Sequence), consider refining your query. \n"